# Summarizing with LangChain and OpenAI

##### 1) When you see `from langchain.chat_models import ChatOpenAI` change it to `from langchain_openai import ChatOpenAI`

##### 2) Also, instead of `answer = chain.run(q)` use `answer = chain.invoke(q)`

In [5]:
import os 
from dotenv import load_dotenv, find_dotenv 

load_dotenv(find_dotenv(), override=True)

True

### 1. Summarizing Using a Basic Prompt

In [6]:
from langchain_openai import ChatOpenAI 
from langchain.schema import AIMessage, HumanMessage, SystemMessage 

In [7]:
text= r"""
Mojo combines the usability of Python with the performance of C, unlocking unparalleled programmability \
of AI hardware and extensibility of AI models.
Mojo is a new programming language that bridges the gap between research and production \ 
by combining the best of Python syntax with systems programming and metaprogramming.
With Mojo, you can write portable code that’s faster than C and seamlessly inter-op with the Python ecosystem.
When we started Modular, we had no intention of building a new programming language. \
But as we were building our platform with the intent to unify the world’s ML/AI infrastructure, \
we realized that programming across the entire stack was too complicated. Plus, we were writing a \
lot of MLIR by hand and not having a good time.
And although accelerators are important, one of the most prevalent and sometimes overlooked "accelerators" \
is the host CPU. Nowadays, CPUs have lots of tensor-core-like accelerator blocks and other AI acceleration \
units, but they also serve as the “fallback” for operations that specialized accelerators don’t handle, \
such as data loading, pre- and post-processing, and integrations with foreign systems. \
"""

In [8]:
messages=[
    SystemMessage(content="You are an expert copywriter with expertize in sumarizing documents."), 
    HumanMessage(content=f"Please provide a short and concise summary of the following text:\n TEXT: {text}")
] 

llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [9]:
llm.get_num_tokens(text)

238

In [10]:
summary_output=llm.invoke(messages) 

print(summary_output) 
print("-"*100) 
print(summary_output.content)

content="Mojo is a new programming language that combines Python's usability with C's performance, enhancing AI hardware programmability and model extensibility. It aims to bridge the gap between research and production by offering faster, portable code that seamlessly integrates with the Python ecosystem. Developed by Modular to simplify ML/AI infrastructure, Mojo recognizes the importance of host CPUs in AI acceleration and aims to streamline programming across the entire stack." response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 278, 'total_tokens': 360}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-ead60542-141d-439f-a6c4-c47e7ec8ae6b-0' usage_metadata={'input_tokens': 278, 'output_tokens': 82, 'total_tokens': 360}
----------------------------------------------------------------------------------------------------
Mojo is a new programming language that combines Python's usability with C

### 2. Summarizing Using Prompt Templates (Length must be Lower than Model's Allowed Token Length)

In [11]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [12]:
template="""   
Write a concise and short summary of the following text: 
TEXT: `{text}`
Translate the summary to {language}. 
"""

prompt=PromptTemplate(
    input_variables=["text", "language"], 
    template=template
    )

In [13]:
llm.get_num_tokens(prompt.format(text=text, language="English"))

261

In [14]:
chain=LLMChain(llm=llm, prompt=prompt) 

response=chain.invoke({"text": text, "language": "English"}) 

print(response) 
print("-"*100) 
print(response["text"])

c:\Users\mpari\anaconda3\envs\LangChain_Laptop_1_venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


{'text': 'Mojo is a new programming language that combines the usability of Python with the performance of C, allowing for unparalleled programmability of AI hardware and extensibility of AI models. It aims to bridge the gap between research and production by offering faster code than C and seamless integration with the Python ecosystem. Mojo was developed by Modular to simplify programming across the entire ML/AI stack and make use of the host CPU as an important accelerator for various operations.', 'language': 'English'}
----------------------------------------------------------------------------------------------------
Mojo is a new programming language that combines the usability of Python with the performance of C, allowing for unparalleled programmability of AI hardware and extensibility of AI models. It aims to bridge the gap between research and production by offering faster code than C and seamless integration with the Python ecosystem. Mojo was developed by Modular to simpli

In [15]:
chain=LLMChain(llm=llm, prompt=prompt) 

response=chain.invoke({"text": text, "language": "Spanish"}) 

print(response) 
print("-"*100) 
print(response["text"])

{'text': 'Mojo es un nuevo lenguaje de programación que combina la usabilidad de Python con el rendimiento de C, desbloqueando una programabilidad sin igual del hardware de IA y la extensibilidad de los modelos de IA. Con Mojo, puedes escribir código portátil más rápido que C y trabajar de forma integrada con el ecosistema de Python. Este lenguaje surge de la necesidad de simplificar la programación en todo el stack de ML/AI, reconociendo la importancia de los aceleradores de CPU en el proceso.', 'language': 'Spanish'}
----------------------------------------------------------------------------------------------------
Mojo es un nuevo lenguaje de programación que combina la usabilidad de Python con el rendimiento de C, desbloqueando una programabilidad sin igual del hardware de IA y la extensibilidad de los modelos de IA. Con Mojo, puedes escribir código portátil más rápido que C y trabajar de forma integrada con el ecosistema de Python. Este lenguaje surge de la necesidad de simplific

### 3. Summarizing using StuffDocumentsChain

In [16]:
from langchain.prompts import PromptTemplate 
from langchain_openai import ChatOpenAI 
from langchain.chains.summarize import load_summarize_chain 
from langchain.docstore.document import Document 

In [17]:
with open("Example_Speech.txt", encoding="utf-8") as f: 
    text=f.read() 

print(text)

docs=[Document(page_content=text)]

I am honored to be with you today at your commencement from one of the finest universities in the world. I never graduated from college. Truth be told, this is the closest I’ve ever gotten to a college graduation. Today I want to tell you three stories from my life. That’s it. No big deal. Just three stories.

The first story is about connecting the dots.

I dropped out of Reed College after the first 6 months, but then stayed around as a drop-in for another 18 months or so before I really quit. So why did I drop out?

It started before I was born. My biological mother was a young, unwed college graduate student, and she decided to put me up for adoption. She felt very strongly that I should be adopted by college graduates, so everything was all set for me to be adopted at birth by a lawyer and his wife. Except that when I popped out they decided at the last minute that they really wanted a girl. So my parents, who were on a waiting list, got a call in the middle of the night asking: “

In [18]:
llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0) 

prompt_template="""   
Write a concise and short summary of the following text. 
TEXT: `{text}`
"""

prompt=PromptTemplate(
    input_variables=["text"], 
    template=prompt_template
    ) 

chain=load_summarize_chain(
    llm=llm, 
    chain_type="stuff", 
    prompt=prompt, 
    verbose=False
)

In [19]:
output_summary=chain.invoke(docs) 

print(output_summary)
print("-"*100) 
print(output_summary["output_text"])

{'input_documents': [Document(page_content='I am honored to be with you today at your commencement from one of the finest universities in the world. I never graduated from college. Truth be told, this is the closest I’ve ever gotten to a college graduation. Today I want to tell you three stories from my life. That’s it. No big deal. Just three stories.\n\nThe first story is about connecting the dots.\n\nI dropped out of Reed College after the first 6 months, but then stayed around as a drop-in for another 18 months or so before I really quit. So why did I drop out?\n\nIt started before I was born. My biological mother was a young, unwed college graduate student, and she decided to put me up for adoption. She felt very strongly that I should be adopted by college graduates, so everything was all set for me to be adopted at birth by a lawyer and his wife. Except that when I popped out they decided at the last minute that they really wanted a girl. So my parents, who were on a waiting lis

### 4. Summarizing Large Documents Using map_reduce (Split Text into Chunks, Summarize each Chunk, and Summarize the Summaries)

##### Used for pieces of text that exceed the given model's (llm's) maximum token length (here, the maximum token length is 4800 tokens)

![alt text](<Screenshot 2024-07-09 152010.png>)

In [20]:
from langchain.prompts import PromptTemplate 
from langchain_openai import ChatOpenAI 
from langchain.chains import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter 

In [21]:
with open("Example_Speech.txt", encoding="utf-8") as f: 
    text=f.read() 

llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [22]:
llm.get_num_tokens(text)

2653

In [23]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=50) 

chunks=text_splitter.create_documents([text])

In [24]:
print(len(chunks))

2


In [25]:
chain_2=load_summarize_chain(
    llm=llm, 
    chain_type="map_reduce", 
    verbose=False
)

In [26]:
output_summary_2=chain_2.invoke(chunks) 

print(output_summary_2) 
print("-"*100) 
print(output_summary_2["output_text"])

{'input_documents': [Document(page_content='I am honored to be with you today at your commencement from one of the finest universities in the world. I never graduated from college. Truth be told, this is the closest I’ve ever gotten to a college graduation. Today I want to tell you three stories from my life. That’s it. No big deal. Just three stories.\n\nThe first story is about connecting the dots.\n\nI dropped out of Reed College after the first 6 months, but then stayed around as a drop-in for another 18 months or so before I really quit. So why did I drop out?\n\nIt started before I was born. My biological mother was a young, unwed college graduate student, and she decided to put me up for adoption. She felt very strongly that I should be adopted by college graduates, so everything was all set for me to be adopted at birth by a lawyer and his wife. Except that when I popped out they decided at the last minute that they really wanted a girl. So my parents, who were on a waiting lis

In [48]:
chain_2.llm_chain.prompt.template # First prompt

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

In [49]:
chain_2.combine_document_chain.llm_chain.prompt.template # Combining/second prompt

'Write a concise summary of the following:\n\n\n"{text}"\n\n\nCONCISE SUMMARY:'

### 5. Summarizing Using map_reduce and Custom Prompts (map_reduce as in what was used above)

In [29]:
map_prompt="""   
Write a short and concise summary of the following: 
TEXT: `{text}` 
CONCISE SUMMARY: 
"""

map_prompt_template=PromptTemplate(
    input_variables=["text"], 
    template=map_prompt
)

In [30]:
combine_prompt="""   
Write a concise summary of the following text that covers the key points. 
Add a title to the summary. 
Start your summary with an INTRODUCTION PARAGRAPH that gives an overview of the topic FOLLOWED 
by BULLET POINTS AND end the summary with a CONCLUSION PHRASE.  
TEXT: `{text}`
"""

combine_prompt_template=PromptTemplate(
    input_variables=["text"], 
    template=combine_prompt
)

In [31]:
# summary_chain=load_summarize_chain(
#     llm=llm,  
#     chain_type="map_reduce", 
#     map_prompt=map_prompt_template, 
#     verbose=False
# )

summary_chain_2=load_summarize_chain(
    llm=llm,  
    chain_type="map_reduce", 
    map_prompt=map_prompt_template, # Initial/first prompt
    combine_prompt=combine_prompt_template, # Combining/second prompt
    verbose=False
)

In [ ]:
# output=summary_chain.invoke(chunks) 

In [ ]:
# print(output)
# print("-"*100) 
# print(output["output_text"])

In [32]:
output_2=summary_chain_2.invoke(chunks) 

In [33]:
print(output_2)
print("-"*100) 
print(output_2["output_text"])

{'input_documents': [Document(page_content='I am honored to be with you today at your commencement from one of the finest universities in the world. I never graduated from college. Truth be told, this is the closest I’ve ever gotten to a college graduation. Today I want to tell you three stories from my life. That’s it. No big deal. Just three stories.\n\nThe first story is about connecting the dots.\n\nI dropped out of Reed College after the first 6 months, but then stayed around as a drop-in for another 18 months or so before I really quit. So why did I drop out?\n\nIt started before I was born. My biological mother was a young, unwed college graduate student, and she decided to put me up for adoption. She felt very strongly that I should be adopted by college graduates, so everything was all set for me to be adopted at birth by a lawyer and his wife. Except that when I popped out they decided at the last minute that they really wanted a girl. So my parents, who were on a waiting lis

### 6. Summarizing Using the Refine CombineDocumentChain

![alt text](<Screenshot 2024-07-09 155910.png>)

##### pip install unstructured
##### pip install pdfminer

In [34]:
from langchain_openai import ChatOpenAI 
from langchain import PromptTemplate 
from langchain.chains.summarize import load_summarize_chain 
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain.document_loaders import UnstructuredPDFLoader 

In [35]:
import os  
from dotenv import load_dotenv, find_dotenv 

load_dotenv(find_dotenv(), override=True)

True

In [36]:
# We can also use PyPDFLoader from langchain.document_loaders 

from langchain.document_loaders import PyPDFLoader 

loader=PyPDFLoader("Example_PDF.pdf") 

data=loader.load()

# loader=UnstructuredPDFLoader("Example_PDF.pdf") 

# data=loader.load() 

print(data)
print("-"*100) 
print(data[0].page_content)

[Document(metadata={'source': 'Example_PDF.pdf', 'page': 0}, page_content='Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models to\nbe superior in qu

In [37]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=100) 

chunks_refine=text_splitter.split_documents(data)

In [38]:
len(chunks_refine)

11

In [39]:
llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [40]:
def print_embedding_cost(texts): 
    import tiktoken 
    enc=tiktoken.encoding_for_model("gpt-3.5-turbo") 
    total_tokens=sum([len(enc.encode(page.page_content)) for page in texts]) 
    print(f"Total Tokens: {total_tokens}") 
    print(f"Embedding Cost in USD: {total_tokens/1000*0.002:.6f}") 

print_embedding_cost(chunks_refine)

Total Tokens: 8236
Embedding Cost in USD: 0.016472


In [41]:
chain=load_summarize_chain(
    llm=llm, 
    chain_type="refine", 
    verbose=True
)

output_summary=chain.invoke(chunks_refine)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.comNoam Shazeer∗
Google Brain
noam@google.comNiki Parmar∗
Google Research
nikip@google.comJakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.comAidan N. Gomez∗†
University of Toronto
aidan@cs.toronto.eduŁukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine trans

In [42]:
print(output_summary) 
print("-"*100) 
print(output_summary["output_text"])

{'input_documents': [Document(metadata={'source': 'Example_PDF.pdf', 'page': 0}, page_content='Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models t

### 7. Summarizing Using Refine and Custom Prompts

In [50]:
prompt_template="""   
Write a concise summary of the following extracting the key information: 
TEXT: `{text}`
CONCISE SUMMARY: 
"""

initial_prompt=PromptTemplate(template=prompt_template, input_variables=["text"])

In [51]:
refine_template="""   
Your job is to produce a final summary. 
I have provided an existing summary up to a certain point: {existing_answer}. 
Please refine the existing summary with some more context below. 
--------------
{text}
--------------
Start the final summary with an INTRODUCTION PARAGRAPH that gives an overview of the topic FOLLOWED 
by BULLET POINTS if possible AND end the summary with a CONCLUSION PHRASE.
"""

refine_prompt=PromptTemplate(
    template=refine_template,
    input_variables=["existing_answer", "text"]
    )

In [55]:
chain_3=load_summarize_chain(
    llm=llm, 
    chain_type="refine", 
    question_prompt=initial_prompt, 
    refine_prompt=refine_prompt, 
    return_intermediate_steps=False
)

In [58]:
output_summary_3=chain_3.invoke(chunks_refine)

In [59]:
print(output_summary_3) 
print("-"*100) 
print(output_summary_3["output_text"])

{'input_documents': [Document(metadata={'source': 'Example_PDF.pdf', 'page': 0}, page_content='Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models t

### 8. Summarizing Using LangChain Agents

In [60]:
from langchain_openai import ChatOpenAI 
from langchain.agents import initialize_agent, Tool 
from langchain.utilities import WikipediaAPIWrapper 

In [ ]:
import os 
from dotenv import load_dotenv, find_dotenv 

load_dotenv(find_dotenv(), override=True)

In [63]:
llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0) 

wikipedia=WikipediaAPIWrapper()

In [64]:
tools=[
    Tool(name="Wikipedia", func=wikipedia.run, description="Useful for when you need to get information for wikipedia about a topic")
]

In [65]:
agent_executor=initialize_agent(
    tools, 
    llm, 
    agent="zero-shot-react-description", 
    verbose=True
)

c:\Users\mpari\anaconda3\envs\LangChain_Laptop_1_venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [66]:
output=agent_executor.invoke("Can you please provide a short summary of George Washington")



> Entering new AgentExecutor chain...
I should use Wikipedia to get a brief summary of George Washington
Action: Wikipedia
Action Input: George Washington
Observation: Page: George Washington
Summary: George Washington (February 22, 1732 – December 14, 1799) was an American Founding Father, military officer, and politician who served as the first president of the United States from 1789 to 1797. Appointed by the Second Continental Congress as commander of the Continental Army in 1775, Washington led Patriot forces to victory in the American Revolutionary War and then served as president of the Constitutional Convention in 1787, which drafted the current Constitution of the United States. Washington has thus become commonly known as the "Father of his Country".
Washington's first public office, from 1749 to 1750, was as surveyor of Culpeper County in the Colony of Virginia. In 1752, he received military training and was granted the rank of major in the Virginia Regiment. During the Fr

In [68]:
print(output)
print("-"*100) 
print(output["output"])

{'input': 'Can you please provide a short summary of George Washington', 'output': 'George Washington was the first president of the United States and a key figure in American history. George Washington Carver was an agricultural scientist and inventor who promoted alternative crops to cotton.'}
----------------------------------------------------------------------------------------------------
George Washington was the first president of the United States and a key figure in American history. George Washington Carver was an agricultural scientist and inventor who promoted alternative crops to cotton.
